# CNN 학습 실습

## Import modules

In [17]:
import tensorflow as tf # tensorflow 중 keras 이용
import numpy as np

from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

## 하이퍼파라미터 정의

In [19]:
EPOCHS = 10

## 네트워크 구조 정의

In [18]:
def MyModel(): # 함수화 해줬음. 지금까진 tensorflow 2.0 방식으로만 실습하고있어서
    # keras 방식으로도 알려줄려고 이번엔 함수화 했음
        return Sequential([Conv2D(32, (3,3), padding='same', activation='relu'), # 28x28x32의 feature map 얻음
                           MaxPool2D(), # 14x14x32
                           Conv2D(64, (3,3), padding='same', activation='relu'), # 14x14x64
                           MaxPool2D(), # 7x7x64
                           Conv2D(128, (3,3), padding='same', activation='relu'), # 7x7x128
                           Flatten(), # 7x7x64 = 6272 
                           Dense(128, activation='relu'), # 128로 줄여줘
                           Dense(10, activation='softmax')]) # fashion mnist는 마지막에 출력이 10개니까
                                           # 32, 64, 128, 256채널 
        # Sequential안에 리스트로 사용할 layer들 다 넣어주면 돼
        # Conv2D(필터개수, 커널 size, )
        # padding='valid': zero-padding 적용x , padding='same': zero-padding 적용, 영상크기 그대로 유지
        # Maxpool2D()는 안에 아무것도 안 넣어주면 알아서 2x2로 작동함

## 데이터 불러오기

In [7]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

# NHWC
x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]
# ...은 앞에 있는 모든 axis에 대해 전부 다 포함한다는 뜻
# np.newaxis는 그 뒤에 axis를 하나 더 붙여준다는 뜻
# 즉, 모든 axis를 포함해서 뒤에 axis 하나 더 붙여준다

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

In [5]:
print(x_train.shape) # (batch 개수, height, width) 
print(x_train[0].shape)

# 3차원으로 되어있는 데이터를 차원을 하나 더 붙여줘야 돼
# CNN 학습할 때 데이터 셋의 구조가 rank=4인 NHWC(batch, height, width, channel)를 필요로 하기때문

(60000, 28, 28)
(28, 28)


In [8]:
print(x_train.shape)
print(x_train[0].shape)

# 4차원 데이터로 바뀌었지

(60000, 28, 28, 1)
(28, 28, 1)


## 모델 생성

In [20]:
model = MyModel()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## 모델 학습

In [21]:
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Train for 1875 steps, validate for 313 steps
Epoch 1/10
1875/1875 [==============================] - 132s 70ms/step - loss: 0.3838 - accuracy: 0.8599 - val_loss: 0.3205 - val_accuracy: 0.8832
Epoch 2/10
1875/1875 [==============================] - 115s 61ms/step - loss: 0.2457 - accuracy: 0.9106 - val_loss: 0.2459 - val_accuracy: 0.9098
Epoch 3/10
1875/1875 [==============================] - 139s 74ms/step - loss: 0.2034 - accuracy: 0.9241 - val_loss: 0.2350 - val_accuracy: 0.9153
Epoch 4/10
1875/1875 [==============================] - 144s 77ms/step - loss: 0.1694 - accuracy: 0.9361 - val_loss: 0.2297 - val_accuracy: 0.9190
Epoch 5/10
1875/1875 [==============================] - 111s 59ms/step - loss: 0.1429 - accuracy: 0.9460 - val_loss: 0.2394 - val_accuracy: 0.9192
Epoch 6/10
1875/1875 [==============================] - 104s 55ms/step - loss: 0.1174 - accuracy: 0.9556 - val_loss: 0.2692 - val_accuracy: 0.9170
Epoch 7/10
1875/1875 [==============================] - 91s 49ms/step - l

- 우리가 사용하던 FC layer만으로 돼있는 DNN보다 성능 더 좋음
- **training loss는 떨어지는데 validation loss는 올라간다는 것은 overfitting이 발생하고있다는 것**
- overfitting 방지하는 방법배웠었지
    - early stopping도 해보고, dense에 drop out도 넣어서 해보고, 모델이 너무 깊은 게 아닌가 해서 모델 줄여서도 해보자
- keras의 Sequential을 이용해 짧은 구현할 때 심플하고 빠르게 구현가능 